In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sl
import networkx as nx
from mlxtend.frequent_patterns import fpgrowth, association_rules, apriori

# Задание 1 «Поиск ассоциативных правил»

#### 1. Загрузите файл «assc_TRANSACTION.csv». В нем три колонки Customer – ID клиента, Product – покупка, Time – временная метка (для задания не нужна). Определите (написав соответствующий код) сколько различных значений принимают переменныеProduct и Customer.

In [ ]:
data_original = pd.read_csv('TRANSACTION.csv')
unique_products=len(data_original.PRODUCT.unique())
unique_customers = len(data_original.CUSTOMER.unique())
unique_products,unique_customers

#### 2. Найдите частые эпизоды с ограничением на размер правила равным 4, с использованием алгоритма и порога на поддержку согласно вашему варианту. (Вариант 1 Apriori, 2%)

Составим таблицу, где столбцами будет имена продуктов, а строками ID покупателей, в ячейках будут булевы значения с информацией о том покупал человек товар или нет

In [ ]:
data = data_original.drop(columns='TIME')
basket = data.pivot_table(index='CUSTOMER', columns='PRODUCT', aggfunc=lambda x: True, fill_value=False)
basket

Отберем частые эпизоды

In [ ]:
min_support = 0.02
frequent_itemsets = apriori(basket, min_support=min_support, use_colnames=True)
frequent_itemsets

Отберем правила с размером <=4

In [ ]:
frequent_itemsets_len4 = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: len(x) <= 4)]
frequent_itemsets_len4

#### 3. Найдите самый большой (где больше всего элементов) частый эпизод, содержащий продукт согласно вашему варианту. Какая у него поддержка? 
Вариант 1 - artichok

In [ ]:
filtered_freq = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: 'artichok' in x)]
largest_episode = filtered_freq.loc[filtered_freq['itemsets'].apply(len).idxmax()]
largest_episode['support'],largest_episode['itemsets']

#### 4. На основе найденных частых эпизодов постройте ассоциативные правила с порогом на достоверность согласно вашему варианту. Найдите правило с максимальным лифтом, содержащем продукт из вашего варианта в левой части правила. Дайте ему письменную словесную интерпретацию, укажите и объясните его числовые показатели: поддержку, достоверность и подъем.
50% согласно варианту

Сформируем правило

In [ ]:
min_confidence = 0.5
rules_by_conf = association_rules(frequent_itemsets, metric='confidence', min_threshold=min_confidence)
rules_by_conf

Найдем правило с максимальным лифтомс продуктом artichok

In [ ]:
filtered_rbc = rules_by_conf[rules_by_conf['antecedents'].apply(lambda x:'artichok' in x)]
best_rule = filtered_rbc.loc[filtered_rbc['lift'].idxmax()]
best_rule

+ Support = 0.098901, это частота, с которой встречаются левая и правая части правила одновременно (A∪B) внутри общего числа транзакция (A∪B/общее число транзакций), то есть в 9.8% корзин содержатся все товары
+ Confidence = 0.733333, это вероятность с которой купят B, при условии покупки A, то есть Support(A∪B)/Support(A), 73% что при покупке (baguette, artichok) купят и (avocado, hering, heineken)
+ Lift = 5.478109, эта величина показывает насколько сильно связаны величины A и B, по сравнению со случайными покупками, lift = 5.5 значит, что вероятность купить (avocado, hering, heineken) при покупке (baguette, artichok) в 5.5 раз выше чем при случаной покупке
-----
 Здесь A (antecedent support) - левая часть правила, B (consequent support) - правая часть правила 

#### 5. Используя только двухместные правила постройте ориентированный граф, где вершины элементы правила, их цвет (или размер) – поддержка элемента (item support), дуги – импликации (ориентированы в направлении от условия к следствию), веса дугu – достоверности.

In [ ]:
filtered_rbc_size2 = rules_by_conf[(rules_by_conf['antecedents'].apply(len) == 1) & (rules_by_conf['consequents'].apply(len) == 1)]
filtered_rbc_size2

In [ ]:
G = nx.DiGraph()
item_support = {item: support for itemset, support in zip(frequent_itemsets['itemsets'], frequent_itemsets['support']) for item in itemset}
for item in item_support:
    G.add_node(item, size=item_support[item] * 10000)
for _, row in filtered_rbc_size2.iterrows():
    A = list(row['antecedents'])[0]
    B = list(row['consequents'])[0]
    confidence = row['confidence']
    G.add_edge(A, B, weight=confidence)
plt.figure(figsize=(16, 9))
pos = nx.spring_layout(G,seed=13)
node_sizes = [G.nodes[node]['size'] for node in G.nodes]
edges = G.edges(data=True)
nx.draw_networkx_nodes(G,pos=pos,node_size=node_sizes)
nx.draw_networkx_edges(G,pos=pos,width=[d['weight'] for (_, _, d) in edges],arrowsize=7)
nx.draw_networkx_labels(G,pos=pos,font_size=8)
plt.show()

#### 6. Для данного графа рассчитайте меры центральности согласно вашему варианту и найдите элемент с самой высокой мерой, а также какую меру имеет продукт из вашего варианта.

Рассчитаем меры центральности для варианта 1 Betweenness

In [ ]:
plt.figure(figsize=(10,6))
metr=nx.betweenness_centrality(G)
metr = dict(sorted(metr.items(), key=lambda x: x[1]))
plt.xticks(rotation = 'vertical')
plt.bar(metr.keys(),metr.values())

Найдем меру artichok и максимальную меру

In [ ]:
metr['artichok'],list(metr.items())[-1]

#### 7. Постройте числовую матрицу со счетчиком числа покупок в ячейках, клиентами по строкам и продуктами по столбцам.

In [ ]:
purchase_matrix = data.pivot_table(index='CUSTOMER', columns='PRODUCT', aggfunc='size', fill_value=0)
purchase_matrix